1. Préparation de l'Environnement et Imports


Cette cellule installe toutes les librairies nécessaires et charge les packages Python.

In [6]:
# Vérification de la GPU et installation des librairies
!nvidia-smi

!pip install -qqq bitsandbytes torch transformers peft accelerate datasets loralib einops trl

import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModelForCausalLM
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorForSeq2Seq,
)

# Fonction utilitaire pour vérifier les paramètres entraînés
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

# Vider le cache du GPU avant de lancer le fine-tuning
torch.cuda.empty_cache()

Mon Nov 10 21:49:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   76C    P0             35W /   70W |    3902MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

2. Chargement et Configuration LoRA pour Qwen

Cette cellule configure spécifiquement le modèle Qwen2.5-0.5B avec les paramètres de quantification et LoRA corrects.

In [7]:
# --- Configuration du Modèle et de la Quantification ---
MODEL_NAME = "Qwen/Qwen2.5-0.5B"

# Configuration BitsAndBytes 4-bit NF4 + BF16 + Double Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

# Chargement du modèle Qwen quantifié
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token # Configuration du pad token


# --- Configuration LoRA (PEFT) pour Qwen ---
# Hyperparamètres R=32, Alpha=64, Dropout=0.05
config_qwen = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    # Modules cibles Qwen/Llama/Mistral
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias='none',
    task_type="CAUSAL_LM"
)

# Application de LoRA
model = get_peft_model(model, config_qwen)
print("Paramètres entraînables après LoRA :")
print_trainable_parameters(model) # Doit afficher environ 5.29% de paramètres entraînables

Paramètres entraînables après LoRA :
trainable params: 17596416 || all params: 332715904 || trainable%: 5.288721034507566


3. Préparation des Données


In [8]:
# --- Chargement et Tokenization des Données (OrangeSum) ---
data = load_dataset("giuliadc/orangesum_5k")["train"]

# ATTENTION : Correction de l'OutOfMemoryError (OOM) en réduisant MAX_LENGTH.
# 2000 tokens sont trop pour une T4, nous utilisons 512 tokens.
MAX_LENGTH = 512

def generate_prompt(data_point):
    # Template de Prompt <human> <assistant>
    return f"<human>: Résumez l'article suivant:\n{data_point['text']}\n<assistant>: {data_point['reference-summary']}"

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point) + tokenizer.eos_token
    # Troncature ajoutée ici avec MAX_LENGTH pour éviter l'OOM
    tokenized_full_prompt = tokenizer(full_prompt, truncation=True, max_length=MAX_LENGTH, return_tensors='pt')

    # On peut ignorer le filtre de longueur si la troncature est appliquée ci-dessus
    if tokenized_full_prompt.input_ids.shape[1] > MAX_LENGTH:
        return None # Devrait être rare avec la troncature

    labels = tokenized_full_prompt.input_ids.clone() # Création des labels

    # Détermination de l'index de début de réponse pour le masquage (-100)
    prompt = full_prompt[:full_prompt.find("<assistant>")] + "<assistant>:"
    tokenized_prompt = tokenizer(prompt, return_tensors='pt')
    end_prompt_idx = tokenized_prompt.input_ids.shape[1] # Index de fin du prompt

    # Masquage des tokens de l'instruction (Completion-Only)
    labels[:, :end_prompt_idx] = -100

    return {
        'input_ids': tokenized_full_prompt.input_ids.flatten(),
        'labels': labels.flatten(),
        'attention_mask': tokenized_full_prompt.attention_mask.flatten(),
    }

# Tokenization et masquage du jeu de données
data = data.shuffle(seed=42).map(generate_and_tokenize_prompt, remove_columns=['id', 'text', 'reference-summary'])
data = data.filter(lambda x: x is not None) # Retirer les None s'il en reste

4. Entraînement du Modèle (Fine-Tuning)

Cette cellule configure le Trainer avec les arguments d'entraînement optimisés et lance le fine-tuning.

In [9]:
# --- Arguments d'Entraînement Optimisés ---
OUTPUT_DIR = "qwen_finetuning_experiments"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1, # Taille de lot minimale
    gradient_accumulation_steps=16, # Grande accumulation pour simuler lot plus grand
    num_train_epochs=2,
    learning_rate=5e-4,
    bf16=True,
    save_total_limit=3,
    logging_steps=20,
    output_dir=OUTPUT_DIR,
    max_steps=200,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.01,
    report_to="tensorboard",
)

# --- Configuration du Trainer ---
trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model), # Padding dynamique
)

model.config.use_cache = False

print("Lancement du Fine-Tuning de Qwen 0.5B (MAX_LENGTH=512)...")
trainer.train()

Lancement du Fine-Tuning de Qwen 0.5B (MAX_LENGTH=512)...


Step,Training Loss
20,1183050.300000
40,4.958800
60,8.367600
80,13.207500
100,2.202700
120,4.433400
140,2.061600
160,674.202800
180,372.690500
200,1.865100


TrainOutput(global_step=200, training_loss=118413.42899118424, metrics={'train_runtime': 2426.9043, 'train_samples_per_second': 1.319, 'train_steps_per_second': 0.082, 'total_flos': 3687627372009216.0, 'train_loss': 118413.42899118424, 'epoch': 0.64})

5. Test et Fusion (Inférence)


Une fois l'entraînement terminé (ou interrompu), cette cellule fusionne les adaptateurs LoRA avec le modèle de base pour un déploiement facile et teste la réponse du modèle.

In [10]:
# --- Définition des Fonctions de Génération et de Réponse ---

def generate_response(prompt: str) -> str:
    """
    Construit le prompt de chat, génère la réponse du modèle et extrait uniquement
    la partie de l'assistant (Completion-Only).
    """
    # 1. Construction du prompt
    full_prompt = f"<human>: {prompt}\n<assistant>:"

    # Réutilisation des configurations de génération
    generation_config = model.generation_config
    generation_config.max_new_tokens = 200
    generation_config.temperature = 0.7
    generation_config.top_p = 0.7
    generation_config.num_return_sequences = 1
    generation_config.pad_token_id = tokenizer.eos_token_id
    generation_config.eos_token_id = tokenizer.eos_token_id
    generation_config.do_sample = True

    device = "cuda:0"
    encoding = tokenizer(full_prompt, return_tensors="pt").to(device)

    # 2. Génération de la réponse
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 3. Extraction de la réponse (après '<assistant>:')
    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)

    # Si le format est trouvé, retourne seulement la réponse de l'assistant
    if response_start != -1:
        return response[response_start + len(assistant_start):].strip()
    return response.strip()


# --- Test du Modèle après Fine-Tuning ---
print("\n" + "="*50)
print("TEST DU MODÈLE APRÈS FINE-TUNING")
print("="*50)

# Article de test en français (Transparence salariale)
article_test = """
Une petite révolution se prépare. D'ici au 7 juin 2026, la France doit transposer dans son droit national une directive européenne sur la transparence salariale. Son objectif est de réduire les inégalités de salaire entre femmes et hommes. Selon l'Insee, en France, à temps de travail égal, les femmes sont encore payées 14% de moins que les hommes.
'À travail égal, rémunération égale. Et pour parvenir à l’égalité de rémunération, il faut de la transparence...', avait déclaré la présidente de la Commission européenne Ursula von der Leyen.
"""

prompt_resume = f"Résumez l'article suivant:\n{article_test}"
print(f"-> Résumé de l'article:\n{generate_response(prompt_resume)}\n")

# Test sur un prompt hors-distribution (hors-sujet)
prompt_ood = "Do you know the reasons as to why people love coffee so much?"
print(f"-> Réponse au prompt OOD:\n{generate_response(prompt_ood)}")


# --- Fusion des Poids LoRA et Sauvegarde Finale ---

print("\n" + "="*50)
print("FUSION ET SAUVEGARDE FINALE")
print("="*50)

# La fusion des adaptateurs LoRA dans le modèle de base
model = model.merge_and_unload()

# Sauvegarde des poids finaux (modèle complet allégé)
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"Modèle fusionné et sauvegardé dans {OUTPUT_DIR}")
print("Le modèle est prêt pour le déploiement sans adaptateurs PEFT.")


TEST DU MODÈLE APRÈS FINE-TUNING
-> Résumé de l'article:
Selon les données de l'Insee, la France doit transposer une directive européenne sur la transparence salariale.

-> Réponse au prompt OOD:
The most popular drink in the world is the coffee. It is a beverage that is enjoyed by people all over the world.

FUSION ET SAUVEGARDE FINALE


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Modèle fusionné et sauvegardé dans qwen_finetuning_experiments
Le modèle est prêt pour le déploiement sans adaptateurs PEFT.
